In [122]:
import random
import time
from IPython.display import clear_output

class Minesweeper():
    """
    Minesweeper game
    """
    def __init__(self):
        self.board = [[]] # Will contain board stae
        self.vis = [[]] # Track which squares are hidden, visible, or flagged
        self.n = -1 # Size of board
        self.n_mines = -1 # Number of mines to use
        self.game_over = False # Track if game is over or not
        self.player_won = False # Track whether or not player has won
        pass
    
    def get_num_adjacent(self, i,j):
        """
        Count the number of mines adjacent to each square.
        
        If square is a mine will return symbol for mine
        """
        diff = [[-1, -1], [-1, 0], [-1, 1], [0, -1], [0, 1],
                [1, -1], [1, 0], [1, 1]]
            
        # If mine, return that
        if self.board[i][j] == '*':
            return '*'
        
        
        count = 0
        for d in diff:
            if (i + d[0] < 0) or (j + d[1] < 0) or (i + d[0] >= self.n) or (j + d[1] >= self.n):
                continue # Out of bounds
            if self.board[i+d[0]][j+d[1]] == '*':
                count += 1                            
                
        return count
        
    def create_world(self, n=10, n_mines=20):
        """
        Create a new world of size n with n_mines. 
        
        Mine will be randomly placed, and number adjacent mines
        will be calculated for each board space.
        
        Mines represented by -1.
        """
        self.n = n
        self.n_mines = n_mines
        self.board = [[0 for i in range(n)] for i in range(n)]
        self.vis = [[0 for i in range(n)] for i in range(n)]
        
        # Randomly place mines until n_mines have been placed. Multiple
        # mines will not be placed on same square
        count = 0        
        while count < n_mines:
            i = random.randint(a=0, b=n-1)
            j = random.randint(a=0, b=n-1)
            if self.board[i][j] == '*':
                continue
            self.board[i][j] = '*'
            count += 1
        
        # Calculate number of mines adjacent to each squares
        for i in range(n):
            for j in range(n):
                self.board[i][j] = self.get_num_adjacent(i,j)
        
    
    def display_world(self):
        """
        Display the current world. 
        
        Unrevealed tiles are displayed as small squares
        """
        clear_output(wait=True)
        
        # Display top row of column numbers
        row1 = "    "
        row2 = "    "
        for i in range(self.n):
            row1 += "{} ".format(i%10)
            row2 += "--"
        
        print(row1)
        print(row2)            
            
        for i in range(self.n):
            cur_row = "{} | ".format(i%10) # Row numbers
            for j in range(self.n):
                if self.vis[i][j] == 1:
                    cur_row += str(self.board[i][j])
                elif self.vis[i][j] == 2:
                    # Square has been flagged
                    cur_row += "F"
                else:
                    # Unicode for small square
                    cur_row += u"\u25A1"
                cur_row += " "
            print(cur_row)
                
    
    def play(self, n=10, n_mines=10):
        """
        This is the main game loop. It will create the world, prompt the user for input,
        update the world, and thn display the world. This will continue until all spaces
        have been revealed for a mine has been hit
        """
        self.create_world(n=n, n_mines=n_mines)  # Initialize world
        self.game_over = False
        self.player_won = False
        self.display_world()
        start = time.time()
        while not self.game_over: # Loop until game is over                        
            i,j,move_type = self.get_move()
            self.update(i,j,move_type)            
            self.display_world()
            self.check_win_condition()
        
        stop = time.time()
        if self.player_won:
            print("Congratulations! You won in {} seconds".format(stop-start))
        else:
            print("Sorry, you hit a mine!")
        
    def get_move(self):
        """
        Keep prompting the player for a move until a valid, non-revealed square is given.
        
        Moves are either a 'pick' move, which reveals the chosen square, or a 
        'flag' move, which places or removes a flag from the given square
        """
        
        while True:
            time.sleep(0.25)
            move_type='pick'
            print("Put an f in front of move to place flag e.g., f 0 2")
            s = input("Please enter input of next move e.g., row, column")
            if s[0] == 'f':
                # If input starts with an f mark it as a flag move and remove the 'f'
                move_type='flag'
                s = s[1:]
            
            x = [int(x) for x in s.replace(',', ' ').split()]
            print(x)
            if len(x) == 2: # Check that two values are supplied
                i = x[0]
                j = x[1]
                if 0 <= i < self.n and 0 <= j < self.n: # Values are on board
                    if self.vis[i][j] == 0: # And values have not been revealed
                        break
                    else:
                        print("This square has already been revealed or flagged")
                        
            print("Please enter two values between 0 and {}".format(self.n))
        return i, j, move_type
    
    def update(self, i, j, move_type):
        """
        Apply the given square at i, j and update the game state. 
        
        This includes revealing the picked square, and if this square is 
        adjacent to 0 mines it will also reveal all adjacent squares.
        """
        if move_type == 'flag':
            if self.vis[i][j] == 0:
                self.vis[i][j] = 2
            else:
                self.vis[i][j] = 0
            return
        
        self.vis[i][j] = 1
        if self.board[i][j] == "*":
            return
            # If a mine is hit then just return            
        else:
            if self.board[i][j] == 0:
                self.reveal_zero_squares(i, j)
                
    def reveal_zero_squares(self, i, j):
        """
        This script is only called if a square with 0 adjacent mines is found
        
        It will uncover the necessary adjacent squares
        """
        # If a square was hit with no adjacent mines, add to stack
        st = [[i,j]] 
        diff = [[-1, -1], [-1, 0], [-1, 1], [0, -1], [0, 1],
            [1, -1], [1, 0], [1, 1]]
        while len(st) > 0: # While stack has items
            i, j = st.pop()
            for d in diff:
                # Check and reveal all adjacent squares, if they are also adjacent to 0
                # mines then add those squares to the stack
                ip = i + d[0]
                jp = j + d[1]
                if (ip < 0) or (jp < 0) or (ip >= self.n) or (jp >= self.n):
                    continue # Out of bounds                                        
                if self.vis[ip][jp] >= 1:
                    continue
                self.vis[ip][jp] = 1
                if self.board[ip][jp] == 0:
                    st.append([ip, jp])                   
       
    def check_win_condition(self):
        """
        Check if player has won. A player has won if there are no non-mine squares
        that have not been revealed. If a mine has been revealed the player has lost
        """
        # Assume game IS over and player has lost,         
        self.game_over = True
                
        for i in range(self.n):
            for j in range(self.n):
                if self.board[i][j] == '*' and self.vis[i][j] == 1:
                    # Player hit mine and has lost
                    self.game_over = True
                    self.player_won = False
                    return
                
                if self.board[i][j] != '*' and self.vis[i][j] == 0:
                    # If the player as a hidden non-mine space then mark game as not over
                    self.game_over = False                    
                    
        # If we're here, all non-mine squares have been revealed and no mines have been hit
        self.player_won = True        
        return
                  

In [123]:
a = Minesweeper()
a.play(n=10, n_mines=8)

    0 1 2 3 4 5 6 7 8 9 
    --------------------
0 | 0 0 0 0 0 0 0 0 0 0 
1 | 0 0 0 0 0 0 0 0 0 0 
2 | 0 0 0 1 1 1 0 0 0 0 
3 | 0 0 0 1 F 1 0 0 0 0 
4 | 0 0 0 1 2 2 1 0 0 0 
5 | 0 0 0 0 1 F 1 0 0 0 
6 | 1 2 1 1 1 1 1 0 0 0 
7 | F 2 F 2 1 F 1 1 0 0 
8 | 1 2 3 F 3 □ F 1 0 0 
9 | 0 0 2 F * □ □ 1 0 0 
Sorry, you hit a mine!
